In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

In [3]:
class TNet(nn.Module):
    def __init__(self, k):
        super(TNet, self).__init__()
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)

    def forward(self, x): # n*2
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = F.leaky_relu(self.conv3(x)) # n*1024
        

class Model(pl.LightningModule):
    def __init__(self, model):
        super(Model, self).__init__()
        self.model = model
        self.learning_rate = 0.001

    def forward(self, x):
        return self.model(x)

In [1]:
import gmsh

def generate_mesh(profile, save_path, visualize=False):
    """
    This function generates mesh for an airfoil from specified options.
    TODO: add more options and finer control.
    """
    gmsh.initialize()
    gmsh.model.add('new model')
    lc = 0.002
    
    airfoil_points = []
    for point in profile:
        x, y = point
        airfoil_points.append(gmsh.model.geo.add_point(x, y, 0, lc))
    airfoil_points.append(airfoil_points[0])
    
    top = gmsh.model.geo.add_point(0, 10, 0, 500*lc)
    center = gmsh.model.geo.add_point(0, 0, 0, 500*lc)
    bottom = gmsh.model.geo.add_point(0, -10, 0, 500*lc)
    arc = gmsh.model.geo.add_circle_arc(top, center, bottom)

    top_right = gmsh.model.geo.add_point(10, 10, 0, 1000*lc)
    bottom_right = gmsh.model.geo.add_point(10, -10, 0, 1000*lc)
    rec = gmsh.model.geo.add_polyline([top, top_right, bottom_right, bottom])
    
    airfoil = gmsh.model.geo.add_spline(airfoil_points)
    
    surface = gmsh.model.geo.add_plane_surface([
        gmsh.model.geo.add_curve_loop([arc, -rec]), # farfield
        gmsh.model.geo.add_curve_loop([airfoil]) 
    ])
    gmsh.model.geo.synchronize()

    airfoil_tag = gmsh.model.add_physical_group(1, [airfoil])
    gmsh.model.set_physical_name(1, airfoil_tag, 'airfoil')
    farfield_tag = gmsh.model.add_physical_group(1, [arc, -rec])
    gmsh.model.set_physical_name(1, farfield_tag, 'farfield')
    surface_tag = gmsh.model.add_physical_group(2, [surface])
    gmsh.model.set_physical_name(2, surface_tag, 'surface')

    gmsh.model.mesh.generate(2)
    gmsh.write(save_path)

    if visualize:
        gmsh.fltk.run()
    gmsh.finalize()



OSError: libGLU.so.1: cannot open shared object file: No such file or directory